In [2]:
import os
import torch
from torch_geometric.data import HeteroData


filename = 'HeteroData_Learnings_v1.pt'
if os.path.exists('./'+filename):
    data = HeteroData.from_dict(torch.load('./'+filename))
    print('loading saved heterodata object')

/home/amos/mambaforge/envs/pyg_torch21/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
data = HeteroData()
data['a'].x = torch.tensor([[1.], [2.], [3.]])

In [12]:
'y' in data['a'].keys()

False

In [6]:
data['skills'].a is None

NameError: name 'data' is not defined

In [ ]:
# sampler for Heterogeneous Graph Transformer

In [4]:
# add self loops so HGT can attend to the self node represenation in previous step
data = data.add_self_loops()

AttributeError: 'HeteroData' has no attribute 'add_self_loops'

In [6]:
data.node_types

['courses_and_programs',
 'qualifications',
 'skills',
 'people',
 'jobs',
 'organizations']

In [14]:
data['qualifications'].num_nodes

1242

In [10]:
data.to_homogeneous().node_type
#value counts for pytorch tensor 
data.to_homogeneous().node_type.unique(return_counts=True)

(tensor([0, 1, 2, 3, 4, 5]),
 tensor([ 55796,   1242, 138698, 293444,  55638,  13613]))

In [3]:
# import transforms
from torch_geometric.transforms import AddSelfLoops

datat = AddSelfLoops()(data)

In [3]:
data['edge_index']

{}

In [1]:
data

NameError: name 'data' is not defined

In [4]:
for x,y in zip(data.edge_types, datat.edge_types):
    print(x)
    print(data[x].edge_index.shape, datat[y].edge_index.shape)

('skills', 'qualification_skill', 'qualifications')
torch.Size([2, 1596]) torch.Size([2, 1596])
('skills', 'course_and_program_skill', 'courses_and_programs')
torch.Size([2, 258099]) torch.Size([2, 258099])
('courses_and_programs', 'course_qualification', 'qualifications')
torch.Size([2, 2099]) torch.Size([2, 2099])
('courses_and_programs', 'course_and_programs_student', 'people')
torch.Size([2, 553454]) torch.Size([2, 553454])
('jobs', 'job_student', 'people')
torch.Size([2, 293444]) torch.Size([2, 293444])
('people', 'supervisor_supervisee', 'people')
torch.Size([2, 217922]) torch.Size([2, 511366])
('people', 'organization_student', 'organizations')
torch.Size([2, 292060]) torch.Size([2, 292060])
('jobs', 'job_job', 'jobs')
torch.Size([2, 18384]) torch.Size([2, 74022])
('skills', 'job_skill', 'jobs')
torch.Size([2, 16289586]) torch.Size([2, 16289586])
('jobs', 'broader_job_job', 'jobs')
torch.Size([2, 54586]) torch.Size([2, 110224])
('qualifications', 'rev_qualification_skill', 'skil